XGBoost model
=====

here we try to implement the xgboost model

In [1]:
import pandas as pd
import numpy as np
%cd ../data
data_original = pd.read_csv('data_cleaned.csv',  delimiter=',', header=0)

/home/raul/uni/data-science/ids2019-m16-traffic-violations/data


# Features

let's add the features and convert them to categorical


In [2]:
data_original.head()


data_x = data_original[['vehicle_brand', 'vehicle_year', 'vehicle_type', 'age_group', 'gender']]
data_x['vehicle_brand'] = pd.Categorical(data_x['vehicle_brand']).codes
data_x['vehicle_year'] = pd.to_numeric(data_x['vehicle_year'])
data_x['vehicle_type'] = pd.Categorical(data_x['vehicle_type']).codes
data_x['age_group'] = pd.Categorical(data_x['age_group']).codes
data_x['gender'] = pd.Categorical(data_x['gender']).codes




data_y = pd.Categorical(data_original['infraction']).codes


/home/raul/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/home/raul/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/raul/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/sta

let's generate a infraction map

In [3]:
infraction_map = pd.DataFrame(data_original['infraction'])
infraction_map['infraction_code'] = pd.Categorical(infraction_map.infraction).codes
infraction_map = infraction_map.drop_duplicates().sort_values('infraction_code').reset_index(drop=True)




infraction_map

,infraction,infraction_code
0,dui,0
1,no_license,1
2,other,2
3,safety,3
4,speeding,4
5,traffic_violation,5
6,unfit_for_road,6


# Balancing the dataset

In [4]:
data_original['infraction'].value_counts()

speeding             181135
traffic_violation     72486
dui                   33591
no_license            28255
safety                25273
other                 21412
unfit_for_road        20141
Name: infraction, dtype: int64

there's a lot of speeding ones which can unbalance the dataset. let's try to balance the dataset by keeping only 70k speeding violations

In [5]:
data_original.head()

speeding_instances = data_original[data_original['infraction'] == 'speeding']
all_other_instances = data_original[data_original['infraction'] != 'speeding']


balanced_speeding_instances = speeding_instances.sample(30000, random_state=0)

data_balanced = pd.concat([balanced_speeding_instances, all_other_instances])

data_balanced['infraction'].value_counts()

traffic_violation    72486
dui                  33591
speeding             30000
no_license           28255
safety               25273
other                21412
unfit_for_road       20141
Name: infraction, dtype: int64

let's create the balanced dataset

In [6]:
data_x_balanced = data_balanced[['vehicle_brand', 'vehicle_year', 'vehicle_type', 'age_group', 'gender']]
data_x_balanced['vehicle_brand'] = pd.Categorical(data_x_balanced['vehicle_brand']).codes
data_x_balanced['vehicle_year'] = pd.to_numeric(data_x_balanced['vehicle_year'])
data_x_balanced['vehicle_type'] = pd.Categorical(data_x_balanced['vehicle_type']).codes
data_x_balanced['age_group'] = pd.Categorical(data_x_balanced['age_group']).codes
data_x_balanced['gender'] = pd.Categorical(data_x_balanced['gender']).codes


data_y_balanced = pd.Categorical(data_balanced['infraction']).codes

/home/raul/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/raul/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/raul/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

# Model with original dataset

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data_x, data_y, random_state=0)

X_train.shape

(286719, 5)

In [8]:
import xgboost as xgb
dtrain = xgb.DMatrix(data=X_train, label=y_train)
dtest = xgb.DMatrix(X_test)
params = {
    'max_depth': 10,
    'objective': 'multi:softmax',  # error evaluation for multiclass training
    'num_class': 7
}
bst = xgb.train(params, dtrain)
y = bst.predict(dtest)

In [9]:
from sklearn.metrics import accuracy_score, classification_report


print(classification_report(y_test, y))


              precision    recall  f1-score   support

           0       0.30      0.12      0.17      8431
           1       0.35      0.13      0.19      7073
           2       0.52      0.55      0.54      5395
           3       0.23      0.03      0.05      6343
           4       0.53      0.95      0.68     45325
           5       0.25      0.03      0.05     18128
           6       0.09      0.00      0.00      4879

    accuracy                           0.51     95574
   macro avg       0.33      0.26      0.24     95574
weighted avg       0.40      0.51      0.40     95574



In [10]:
df_y = pd.DataFrame(y)

df_y[0].value_counts()

4.0    80767
2.0     5731
0.0     3418
1.0     2623
5.0     2214
3.0      713
6.0      108
Name: 0, dtype: int64

# Model with balanced dataset

In [11]:
from sklearn.model_selection import train_test_split

# let's use this method, but ignore the balanced test datasets - 
# going to verify instead using the original one
X_train_bal, X_test_bal, y_train_bal, y_test_bal = train_test_split(data_x_balanced,
                                                                    data_y_balanced, 
                                                                    random_state=0)

X_test_bal.shape

(57790, 5)

In [12]:
import xgboost as xgb
dtrain_bal = xgb.DMatrix(data=X_train_bal, label=y_train_bal)
dtest = xgb.DMatrix(X_test)
params = {
    'max_depth': 10,
    'objective': 'multi:softmax',  # error evaluation for multiclass training
    'num_class': 7
}
bst_bal = xgb.train(params, dtrain_bal)


In [13]:
from sklearn.metrics import accuracy_score, classification_report

y_bal_pred = bst_bal.predict(dtest)
print(classification_report(y_test, y_bal_pred))


              precision    recall  f1-score   support

           0       0.22      0.29      0.25      8431
           1       0.28      0.22      0.25      7073
           2       0.43      0.53      0.48      5395
           3       0.24      0.06      0.10      6343
           4       0.78      0.03      0.06     45325
           5       0.20      0.76      0.32     18128
           6       0.20      0.01      0.02      4879

    accuracy                           0.24     95574
   macro avg       0.34      0.27      0.21     95574
weighted avg       0.50      0.24      0.16     95574



In [14]:
df_y_bal = pd.DataFrame(y_bal_pred)

df_y_bal[0].value_counts()

5.0    68522
0.0    10834
2.0     6744
1.0     5638
4.0     1910
3.0     1611
6.0      315
Name: 0, dtype: int64